In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'project-mimic-430923'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

In [4]:
# 1. Get basic patient info (admissions + demographics (excluding marital_status) + death status)
patient_info_query = """
SELECT adm.subject_id, adm.hadm_id, adm.admittime, adm.dischtime, pat.gender, pat.anchor_age AS age, adm.race, adm.hospital_expire_flag
FROM `physionet-data.mimiciv_hosp.admissions` AS adm
JOIN `physionet-data.mimiciv_hosp.patients` AS pat
ON adm.subject_id = pat.subject_id
WHERE adm.admittime IS NOT NULL
"""
patient_info_df = run_query(patient_info_query)

patient_info_df['in_hospital_death'] = patient_info_df['hospital_expire_flag'] == 1

patient_info_df = patient_info_df.drop(columns=['hospital_expire_flag'])

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [5]:
patient_info_df

,subject_id,hadm_id,admittime,dischtime,gender,age,race,in_hospital_death
0,14962874,22614141,2165-10-15 15:59:00,2165-10-18 16:47:00,M,89,WHITE,False
1,13241600,22574379,2162-10-29 15:36:00,2162-10-30 16:00:00,M,89,WHITE,False
2,12945423,20185299,2122-07-08 05:40:00,2122-07-19 15:15:00,M,89,WHITE,False
3,16014534,21633959,2176-03-03 14:33:00,2176-03-11 13:57:00,F,89,WHITE,False
4,16014534,27188276,2176-03-27 11:53:00,2176-03-31 14:10:00,F,89,WHITE,False
...,...,...,...,...,...,...,...,...
431226,12545126,27135177,2187-11-01 15:47:00,2187-11-02 09:45:00,M,91,WHITE,False
431227,17579295,26848807,2162-11-16 07:15:00,2162-11-19 14:27:00,M,91,WHITE,False
431228,18563244,26485584,2136-08-18 14:48:00,2136-08-19 11:00:00,F,91,WHITE,False
431229,14865704,29199399,2156-11-12 13:59:00,2156-11-13 14:00:00,F,91,OTHER,False


In [6]:
# Count number of patients
num_patients = patient_info_df['subject_id'].nunique()
print(f"Number of patients: {num_patients}")

Number of patients: 180733


In [7]:
# 2. Get item IDs for lipase tests
lipase_item_query = """
SELECT itemid, label, fluid
FROM `physionet-data.mimiciv_hosp.d_labitems`
WHERE LOWER(label) LIKE '%lipase%'
"""
lipase_items_df = run_query(lipase_item_query)
print(lipase_items_df)
lipase_itemids = [50956, 50844, 51055, 51036] # blood and other fluid # Lipase item IDs [50956]# only blood

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   itemid               label             fluid
0   50956              Lipase             Blood
1   50844     Lipase, Ascites           Ascites
2   51055     Lipase, Pleural           Pleural
3   51036  Lipase, Body Fluid  Other Body Fluid


In [8]:
# 3. Retrieve lipase values
lipase_values_query = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_hosp.labevents`
WHERE itemid IN ({', '.join(map(str, lipase_itemids))})
ORDER BY subject_id, charttime
"""
lipase_values_df = run_query(lipase_values_query)

# Merge lipase values with patient info and clean data
lipase_values_df = pd.merge(lipase_values_df, patient_info_df, on=['subject_id', 'hadm_id'], how='left')
lipase_values_df.dropna(subset=['age', 'lipase_level'], inplace=True)

# Identify patients with high lipase levels
def check_lipase(row):
    upper_limit = 140 if row['age'] < 60 else 151
    return row['lipase_level'] >= 3 * upper_limit

high_lipase_df = lipase_values_df[lipase_values_df.apply(check_lipase, axis=1)]

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [9]:
high_lipase_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death
85,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False
170,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False
362,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
363,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
427,10021357,25937617,2144-12-30 06:55:00,1249.0,2144-12-27 19:41:00,2145-01-04 19:54:00,F,91,WHITE,False
...,...,...,...,...,...,...,...,...,...,...
229546,19990545,23106222,2139-10-14 06:11:00,527.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False
229547,19990545,23106222,2139-10-15 04:40:00,753.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False
229548,19990545,23106222,2139-10-16 04:50:00,650.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False
229672,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False


In [10]:
# 3. Retrieve lipase values icu

lipase_itemids_icu = "225672"  # Lipase item ID ICU

lipase_values_query_icu = f"""
SELECT subject_id, hadm_id, charttime, valuenum AS lipase_level
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid IN ({lipase_itemids_icu})
ORDER BY subject_id, charttime
"""
lipase_values_df_icu = run_query(lipase_values_query_icu)

# Merge lipase values with patient info and clean data
lipase_values_df_icu = pd.merge(lipase_values_df_icu, patient_info_df, on=['subject_id', 'hadm_id'], how='left')
lipase_values_df_icu.dropna(subset=['age', 'lipase_level'], inplace=True)

# Identify patients with high lipase levels in ICU
def check_lipase(row):
    upper_limit = 140 if row['age'] < 60 else 151
    return row['lipase_level'] >= 3 * upper_limit

high_lipase_df_icu = lipase_values_df_icu[lipase_values_df_icu.apply(check_lipase, axis=1)]

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [11]:
high_lipase_df_icu

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death
1,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False
15,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
16,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
55,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False
56,10036086,28728587,2196-05-28 03:39:00,777.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False
...,...,...,...,...,...,...,...,...,...,...
17140,19882958,29628147,2182-08-31 03:10:00,787.0,2182-08-29 17:25:00,2182-09-03 13:50:00,M,83,WHITE,False
17167,19899716,21665899,2143-09-03 17:16:00,587.0,2143-09-03 12:59:00,2143-09-08 18:00:00,M,62,BLACK/AFRICAN AMERICAN,False
17178,19901341,23906609,2169-08-10 06:48:00,588.0,2169-08-06 14:52:00,2169-09-05 14:58:00,F,55,WHITE,False
17189,19907884,24707264,2181-01-24 04:42:00,789.0,2181-01-23 21:32:00,2181-02-03 13:30:00,F,38,WHITE,False


In [12]:
high_lipase_df_combined = pd.concat([high_lipase_df, high_lipase_df_icu])

# Drop duplicate rows
high_lipase_df_unique = high_lipase_df_combined.drop_duplicates()

In [13]:
# Count number of patients with lipase levels recorded
num_patients = high_lipase_df_unique['subject_id'].nunique()
print(f"Number of unique patients with high lipase levels: {num_patients}")

Number of unique patients with high lipase levels: 2106


In [14]:
high_lipase_df_unique

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death
85,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False
170,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False
362,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
363,10017531,22580355,2159-09-23 04:30:00,977.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False
427,10021357,25937617,2144-12-30 06:55:00,1249.0,2144-12-27 19:41:00,2145-01-04 19:54:00,F,91,WHITE,False
...,...,...,...,...,...,...,...,...,...,...
13088,17509107,25893958,2122-06-07 02:05:00,999999.0,2122-06-05 00:17:00,2122-06-30 12:00:00,M,49,WHITE,False
14505,18341278,28924376,2141-05-25 23:34:00,493.0,2141-05-25 21:27:00,2141-05-29 17:08:00,M,56,BLACK/AFRICAN AMERICAN,False
15970,19201291,25546590,2184-01-15 02:08:00,492.0,2184-01-04 23:43:00,2184-01-22 13:15:00,M,41,BLACK/AFRICAN AMERICAN,False
15971,19201291,25546590,2184-01-15 05:51:00,529.0,2184-01-04 23:43:00,2184-01-22 13:15:00,M,41,BLACK/AFRICAN AMERICAN,False


In [16]:
patient_info_df['admittime'] = pd.to_datetime(patient_info_df['admittime'])
patient_info_df['dischtime'] = pd.to_datetime(patient_info_df['dischtime'])
patient_info_df['length_of_stay'] = (patient_info_df['dischtime'] - patient_info_df['admittime']).dt.total_seconds() / (60 * 60 * 24)

In [17]:
high_lp_df = pd.merge(high_lipase_df_unique, patient_info_df[['subject_id', 'hadm_id', 'length_of_stay']], on=['subject_id', 'hadm_id'], how='left')

In [19]:
def select_row(group):
    # Check if there's any row with hospital_expire_flag = 1
    if (group['in_hospital_death']).any():
        # Return the first row with hospital_expire_flag = 1
        return group[group['in_hospital_death']].iloc[0]
    else:
        # Otherwise, return the row with the maximum length_of_stay
        return group.loc[group['length_of_stay'].idxmax()]

# Apply the function to each group and reset the index
high_lp_filtered_df = high_lp_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)

<ipython-input-19-dde7ba27835e>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_lp_filtered_df = high_lp_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [20]:
high_lp_filtered_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556
1,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,6.654167
2,10017531,22580355,2159-09-22 20:56:00,1164.0,2159-09-22 19:30:00,2159-10-24 13:40:00,M,63,WHITE,False,31.756944
3,10021357,25937617,2144-12-30 06:55:00,1249.0,2144-12-27 19:41:00,2145-01-04 19:54:00,F,91,WHITE,False,8.009028
4,10036086,28728587,2196-05-26 09:25:00,677.0,2196-05-20 02:47:00,2196-06-12 11:42:00,M,57,WHITE,False,23.371528
...,...,...,...,...,...,...,...,...,...,...,...
2101,19970491,22119205,2131-02-11 03:06:00,487.0,2131-02-10 18:25:00,2131-02-17 14:15:00,M,55,WHITE,False,6.826389
2102,19977727,23685838,2170-06-12 07:20:00,528.0,2170-06-10 22:54:00,2170-06-12 14:37:00,F,64,WHITE,False,1.654861
2103,19990545,23106222,2139-10-06 08:30:00,1886.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,24.690278
2104,19996968,29843339,2125-01-23 06:25:00,508.0,2125-01-20 21:17:00,2125-01-23 14:42:00,M,32,BLACK/AFRICAN AMERICAN,False,2.725694


In [21]:
high_lp_filtered_df.to_csv('High_Lipase_Dataset.csv', encoding='utf-8', index = False)

In [30]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in high_lp_filtered_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(high_lp_filtered_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
2    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
3    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
4    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   

            dischtime gender  age   race  in_hospital_death  length_of_stay  \
0 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
1 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
2 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
3 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   
4 2159-03-06 16:51:00      F   64  WHITE              False       14.130556   

  icd_code  seq_num  icd_version  
0  

In [31]:
merged_comorbidities_df.to_csv('High_Lipase_Comorbidities_Dataset.csv', encoding='utf-8', index = False)

In [22]:
ap_icd_query = f"""
SELECT subject_id, hadm_id, icd_code
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_code LIKE 'K85%'
ORDER BY subject_id
"""
ap_icd_df = run_query(ap_icd_query)

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [23]:
ap_icd_df

,subject_id,hadm_id,icd_code
0,10002807,28464737,K8590
1,10004606,29242151,K8510
2,10006431,24638489,K8580
3,10008816,22267961,K859
4,10012206,23961896,K8510
...,...,...,...
1872,19939903,26616095,K851
1873,19968351,23732375,K8590
1874,19972266,24170910,K8590
1875,19990545,23106222,K8590


In [24]:
ap_icd_df['subject_id'].nunique()

1401

In [25]:
merged_ap_icd_lp_df = pd.merge(high_lipase_df_unique, ap_icd_df, on=['subject_id', 'hadm_id'], how='inner')

# Merge length of stay into the dataset
merged_ap_icd_lp_df = pd.merge(merged_ap_icd_lp_df, patient_info_df[['subject_id', 'hadm_id', 'length_of_stay']], on=['subject_id', 'hadm_id'], how='left')

In [26]:
merged_ap_icd_lp_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,icd_code,length_of_stay
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,K8510,14.130556
1,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,K8580,6.654167
2,10110843,20574972,2164-07-16 03:13:00,4175.0,2164-07-14 17:29:00,2164-07-18 10:30:00,F,72,WHITE,False,K8580,3.709028
3,10150465,27749560,2155-06-09 09:05:00,8470.0,2155-06-09 04:45:00,2155-06-19 17:07:00,F,67,WHITE,False,K8500,10.515278
4,10159832,22502935,2117-04-04 06:48:00,1870.0,2117-04-03 15:35:00,2117-04-08 19:18:00,F,49,WHITE,False,K8510,5.154861
...,...,...,...,...,...,...,...,...,...,...,...,...
759,19990545,23106222,2139-10-14 06:11:00,527.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,K8590,24.690278
760,19990545,23106222,2139-10-15 04:40:00,753.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,K8590,24.690278
761,19990545,23106222,2139-10-16 04:50:00,650.0,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,K8590,24.690278
762,11741025,28221700,2135-04-29 17:20:00,2872.0,2135-04-29 12:52:00,2135-05-02 14:40:00,M,91,WHITE,False,K851,3.075000


In [27]:
# Count the number of patients with high lipase levels who had a CT scan
num_ap_icd_lp_df = merged_ap_icd_lp_df['subject_id'].nunique()
print(f"Number of patients with high lipase levels who had a CT scan: {num_ap_icd_lp_df}")

# Count the number of rows in the dataset
num_rows = merged_ap_icd_lp_df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Number of patients with high lipase levels who had a CT scan: 429
Number of rows in the dataset: 764


In [28]:
high_lp_ap_filtered_df = merged_ap_icd_lp_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)

<ipython-input-28-e805d225ae13>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_lp_ap_filtered_df = merged_ap_icd_lp_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [29]:
high_lp_ap_filtered_df

,subject_id,hadm_id,charttime,lipase_level,admittime,dischtime,gender,age,race,in_hospital_death,icd_code,length_of_stay
0,10004606,29242151,2159-02-20 18:30:00,1222.0,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,K8510,14.130556
1,10006431,24638489,2129-01-23 23:36:00,508.0,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,K8580,6.654167
2,10110843,20574972,2164-07-16 03:13:00,4175.0,2164-07-14 17:29:00,2164-07-18 10:30:00,F,72,WHITE,False,K8580,3.709028
3,10150465,27749560,2155-06-09 09:05:00,8470.0,2155-06-09 04:45:00,2155-06-19 17:07:00,F,67,WHITE,False,K8500,10.515278
4,10159832,22502935,2117-04-04 06:48:00,1870.0,2117-04-03 15:35:00,2117-04-08 19:18:00,F,49,WHITE,False,K8510,5.154861
...,...,...,...,...,...,...,...,...,...,...,...,...
424,19811321,23611931,2187-03-09 07:05:00,923.0,2187-03-08 05:23:00,2187-03-19 11:48:00,M,58,WHITE,False,K8511,11.267361
425,19829765,21908728,2137-05-21 05:25:00,932.0,2137-05-20 21:45:00,2137-05-22 13:43:00,F,57,WHITE,False,K8510,1.665278
426,19876636,26288280,2173-08-04 08:30:00,507.0,2173-08-03 14:26:00,2173-08-05 17:45:00,M,56,WHITE,False,K8520,2.138194
427,19882958,29628147,2182-08-31 03:10:00,787.0,2182-08-29 17:25:00,2182-09-03 13:50:00,M,83,WHITE,False,K8510,4.850694


In [32]:
high_lp_ap_filtered_df.to_csv('High_LP_AP_ICD_Dataset.csv', encoding='utf-8', index = False)

In [33]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in high_lp_ap_filtered_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(high_lp_ap_filtered_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           charttime  lipase_level           admittime  \
0    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
1    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
2    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
3    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   
4    10004606  29242151 2159-02-20 18:30:00        1222.0 2159-02-20 13:43:00   

            dischtime gender  age   race  in_hospital_death icd_code_x  \
0 2159-03-06 16:51:00      F   64  WHITE              False      K8510   
1 2159-03-06 16:51:00      F   64  WHITE              False      K8510   
2 2159-03-06 16:51:00      F   64  WHITE              False      K8510   
3 2159-03-06 16:51:00      F   64  WHITE              False      K8510   
4 2159-03-06 16:51:00      F   64  WHITE              False      K8510   

   length_of_stay icd_code_y  seq_num  icd_version  
0       14.1305

In [34]:
merged_comorbidities_df.to_csv('High_LP_AP_ICD_Comorbidities_Dataset.csv', encoding='utf-8', index = False)

In [35]:
# Count the number of unique patients who died in the hospital
num_patients_with_death_info = merged_comorbidities_df[merged_comorbidities_df['in_hospital_death'] == True][['subject_id', 'hadm_id']].drop_duplicates().shape[0]
print(f"Number of unique patients who died in the hospital: {num_patients_with_death_info}")

Number of unique patients who died in the hospital: 22


In [36]:
# 1. Handle missing values
missing_data = merged_comorbidities_df.isnull().sum()
print(missing_data)

subject_id           0
hadm_id              0
charttime            0
lipase_level         0
admittime            0
dischtime            0
gender               0
age                  0
race                 0
in_hospital_death    0
icd_code_x           0
length_of_stay       0
icd_code_y           0
seq_num              0
icd_version          0
dtype: int64
